In [4]:
# 策略模式 一组实现相同接口的算法，插入上下文中运行，例如订单-打折算法
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity
  
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self,'__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total
  
  def due(self):
    if self.promotion is None:
      discount = 0 
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount
  
  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

 
class Promotion(ABC):
  
  @abstractmethod
  def discount(self, order):
    """return a discount"""

class FidelityPromo(Promotion):

  def discount(self, order):
    return order.total()*0.05 if order.customer.fidelity >= 1000 else 0
  
class BulkItemPromo(Promotion):
  
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total()*0.1
    return discount
    

class LargeOrderPromo(Promotion):
  
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total()*0.07
    return 0
  
joe = Customer('Joe Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
  LineItem('banana', 4 , 0.5),
  LineItem('apple',10,1.5),
  LineItem('paper',5,5.0)
]
print(Order(joe, cart, FidelityPromo()))
print(Order(ann,cart,FidelityPromo()))
        

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>


In [8]:
# 使用只有一个函数的抽象类来实现打折算法有点太多余，可以把函数直接当参数传入类
# 其他不变，改写Order中的due函数
def due(self):
  if self.promotion is None:
    discount = 0
  else:
    # 直接调用作为属性传入的promotion
    discount = self.promotion(self)
  return self.total() - discount

# 打折算法就直接实现一个函数即可
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * .1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * .07
  return 0


In [ ]:
# 通常，会给用户一个最佳折扣，实现如下，很简单，但是需要自己维护promos列表
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [ ]:
# 可以使用global函数寻找本模块中的promo函数，自动加到promos列表中
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

# 另一种做法是把所有promo函数除了best_promo都放到一个promotions模块中，然后用inspect模块获取
# promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]